## Assignment 7

### Assignment 7.1

In this part of the assignment, you will partition a dataset using different strategies. You will use the routes.parquet dataset you created in a previous assignment. For this dataset, the key for each route will be the three-letter source airport code concatenated with the three-letter destination airport code and the two-letter airline. For instance, a route from Omaha Eppley Airfield (OMA) to Denver International Airport (DEN) on American Airlines (AA) has a key of OMADENAA.

#### a.

Start by loading the dataset from the previous assignment using Pandas's read_parquet method. Next, add the concatenated key then using Panda's apply method to create a new column called key. For this part of the example, we will create 16 partitions so that we can compare it to the partitions we create from hashed keys in the next part of the assignment. The partitions are determined by the first letter of the composite key using the following partitions.

In [41]:
import os
import json
from pathlib import Path
import gzip
import hashlib
import shutil

import pandas as pd
import pygeohash
import s3fs

endpoint_url='https://storage.budsc.midwest-datascience.com'

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')

if results_dir.exists():
    shutil.rmtree(results_dir)
results_dir.mkdir(parents=True, exist_ok=True)

def read_jsonl_data():
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    with s3.open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            records = [json.loads(line) for line in f.readlines()]
        

    return records

def flatten_record(record):
    flat_record = dict()
    for key, value in record.items():
        if key in ['airline', 'src_airport', 'dst_airport']:
            if isinstance(value, dict):
                for child_key, child_value in value.items():
                    flat_key = '{}_{}'.format(key, child_key)
                    flat_record[flat_key] = child_value
        else:
            flat_record[key] = value
    
    return flat_record

def create_flattened_dataset():
    records = read_jsonl_data()
    parquet_path = results_dir.joinpath('routes-flattened.parquet')
    return pd.DataFrame.from_records([flatten_record(record) for record in records])

df = create_flattened_dataset()
df['key'] = df['src_airport_iata'].astype(str) + df['dst_airport_iata'].astype(str) + df['airline_iata'].astype(str)

In [43]:
df.head()

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_longitude,dst_airport_altitude,dst_airport_timezone,dst_airport_dst,dst_airport_tz_id,dst_airport_type,dst_airport_source,codeshare,equipment,key
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],AERKZN2B
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFKZN2B
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,43.081902,1054.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFMRV2B
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],CEKKZN2B
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,82.650703,365.0,7.0,N,Asia/Krasnoyarsk,airport,OurAirports,False,[CR2],CEKOVB2B


In [66]:
def first(string):
    kv = string[0]
    partitions = (
    ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
    ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
    ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
    ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
)

    for i in partitions:
        if kv in i:
            if i[0] == i[1]:
                return "{0}".format(i[0])
            else:
                return "{0}-{1}".format(i[0], i[1])
    

df['kv_key'] = df['key'].apply(first)

In [67]:
df.head()

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_altitude,dst_airport_timezone,dst_airport_dst,dst_airport_tz_id,dst_airport_type,dst_airport_source,codeshare,equipment,key,kv_key
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],AERKZN2B,A
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFKZN2B,A
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,1054.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFMRV2B,A
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],CEKKZN2B,C-D
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,365.0,7.0,N,Asia/Krasnoyarsk,airport,OurAirports,False,[CR2],CEKOVB2B,C-D


In [47]:
os.mkdir(os.path.join(current_dir, 'results/kv'))

In [69]:
df.to_parquet(path = os.path.join(current_dir, 'results/kv'), partition_cols = ['kv_key'])

#### b.

Next, we are going to partition the dataset again, but this time we will partition by the hash value of the key. The following is a function that will create a SHA256 hash of the input key and return a hexadecimal string representation of the hash.

In [70]:
import hashlib

def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return m.hexdigest()

We will partition the data using the first character of the hexadecimal hash. As such, there are 16 possible partitions. Create a new column called hashed that is a hashed value of the key column. Next, create a partitioned dataset based on the first character of the hashed key and save the results to results/hash. The directory should contain the following folders.

In [71]:
os.mkdir(os.path.join(results_dir, 'hash'))

In [72]:
df['hashed'] = df['key'].apply(hash_key)

In [73]:
df.head()

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_timezone,dst_airport_dst,dst_airport_tz_id,dst_airport_type,dst_airport_source,codeshare,equipment,key,kv_key,hashed
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],AERKZN2B,A,652cdec02010381f175efe499e070c8cbaac1522bac59a...
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFKZN2B,A,9eea5dd88177f8d835b2bb9cb27fb01268122b635b241a...
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFMRV2B,A,161143856af25bd4475f62c80c19f68936a139f653c1d3...
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],CEKKZN2B,C-D,39aa99e6ae2757341bede9584473906ef1089e30820c90...
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,7.0,N,Asia/Krasnoyarsk,airport,OurAirports,False,[CR2],CEKOVB2B,C-D,143b3389bce68eea3a13ac26a9c76c1fa583ec2bd26ea8...


In [76]:
def hash_first(string):
    return string[0]

In [81]:
df['hash_key'] = df['hashed'].apply(hash_first)

In [86]:
df.to_parquet(path = os.path.join(results_dir, 'hash'), partition_cols = ['hash_key'])

#### c.

Finally, we will simulate multiple geographically distributed data centers. For this example, we will assume we have three data centers located in the western, central, and eastern United States. Google lists the locations of their data centers and we will use the following locations for our three data centers.

In [124]:
df['src_airport_geohash'] = df.apply(
    lambda row: pygeohash.encode(row.src_airport_latitude, row.src_airport_longitude), axis = 1
)

def determine_location(src_airport_geohash):
    locations = dict(
        central = pygeohash.encode(41.1544433, -96.0422378),
        # TODO: add west and east
        west = pygeohash.encode(45.5945645, -121.1786823),
        east = pygeohash.encode(39.08344, -77.6497145)
    )
    
    distances = []
    
    #TODO: a list of centers and distances using the          pygeohash.geohash_haversine_distance function
    
    for i in locations:
        dist = pygeohash.geohash_haversine_distance(str(src_airport_geohash), str(locations[i]))
        distances.append((dist, i))
    
    distances.sort()
    return distances[0][1]

df['location'] = df['src_airport_geohash'].apply(determine_location)

df.to_parquet('results/geo', partition_cols = ['location'])

#### d.

Create a Python function that takes as input a list of keys and the number of partitions and returns a list of keys sorted into the specified number of partitions. The partitions should be roughly equal in size. Furthermore, the partitions should have the property that each partition contains all the keys between the least key in the partition and the greatest key in the partition. In other words, the partitions should be ordered.

In [170]:
def balance_partitions(keys, num_partitions):
    partitions = []
    
    keys = keys.sort_values()
    
    average = len(keys) / float(num_partitions)
    last = 0.0
    
    while last < len(keys):
        for i in range(num_partitions):
            partitions.append(('Partition {}'.format(i + 1), list(keys[int(last):int(last + average)])))
            last += average
    
    return partitions

In [193]:
balance_partitions(df['key'], 10)

[('Partition 1',
  ['AAEALGAH',
   'AAECDGAH',
   'AAEISLAH',
   'AAELYSAH',
   'AAEMRSAH',
   'AAEMRSZI',
   'AAEORNAH',
   'AAEORYAH',
   'AAEORYZI',
   'AALAARBA',
   'AALAGPDY',
   'AALALCDY',
   'AALAMSAZ',
   'AALAMSKL',
   'AALARNSK',
   'AALBCNIB',
   'AALBCNVY',
   'AALBLLDX',
   'AALBLLSK',
   'AALBLLTK',
   'AALCPHDY',
   'AALCPHSK',
   'AALISLTK',
   'AALLGWDY',
   'AALOSLBA',
   'AALOSLM3',
   'AALOSLSK',
   'AALPMIDY',
   'AALSVGDX',
   'AANCCJIX',
   'AANPEWNL',
   'AAQDMES7',
   'AAQLEDSU',
   'AAQSVOSU',
   'AARAALBA',
   'AARAGPFR',
   'AARBMABA',
   'AARCPHSK',
   'AARGOTBA',
   'AAROSLBA',
   'AARPMIFR',
   'AARSTNFR',
   'AATURCCZ',
   'AATURCGS',
   'AAXPOJAD',
   'AAYSAHFO',
   'ABADMES7',
   'ABAIKTnan',
   'ABANSKY7',
   'ABASVOSU',
   'ABDMHDB9',
   'ABDMHDIR',
   'ABDSYZEP',
   'ABDTHRB9',
   'ABDTHREP',
   'ABDTHRIR',
   'ABEATLAF',
   'ABEATLDL',
   'ABEATLKL',
   'ABECLTAA',
   'ABECLTUS',
   'ABEDTWDL',
   'ABEMYRG4',
   'ABEORDUA',
   'ABEPGDG4',
   'ABE